# Validate Analysis Results

This notebook compares analysis outputs between a **production run** (baseline) and a **test run** to validate that the test produced expected results.

**Endpoints compared:**
- Statistics (`/stats`)
- EP Metrics (`/ep`)
- Event Loss Table (`/elt`)
- Period Loss Table (`/plt`) - HD analyses only

**Fields compared per endpoint:**

| Endpoint | Fields |
|----------|--------|
| Statistics | epType, purePremium, totalStdDev, cv, netPurePremium, activation, exhaustion, totalLossRatio, limit, premium, netStdDev, exhaustAllReinstatements |
| EP Metrics | epType, value (contains returnPeriods and positionValues arrays) |
| ELT | eventId, sourceId, positionValue, stdDevI, stdDevC, expValue, rate, peril, region, oepWUC |
| PLT | periodId, eventId, weight, eventDate, lossDate, positionValue, peril, region |

## 1. Setup & Imports

In [1]:
%load_ext autoreload
%autoreload 2

from helpers.analysis_results_validator import (
    AnalysisResultsValidator,
    print_validation_summary,
    print_validation_details,
)

validator = AnalysisResultsValidator()
print("Setup complete.")

Setup complete.


## 2. Configuration

Enter the **appAnalysisId** values for the production and test analyses to compare.

The `appAnalysisId` is the ID shown in the Moody's RiskModeler UI (e.g., 35810).

In [2]:
# Analysis IDs to compare
PRODUCTION_APP_ANALYSIS_ID = 1665  # Replace with your production analysis ID
TEST_APP_ANALYSIS_ID = 4419        # Replace with your test analysis ID

# Perspective code: 'GR' (Gross), 'GU' (Ground-Up), 'RL' (Reinsurance Layer)
PERSPECTIVE_CODE = 'GR'

# Include PLT comparison? (only for HD analyses)
INCLUDE_PLT = False

# Comparison settings
RELATIVE_TOLERANCE = 1e-4  # For floating-point comparison
MAX_DIFFERENCES_TO_SHOW = 50  # Limit output for large datasets

## 3. Run Validation

In [3]:
result = validator.validate(
    production_app_analysis_id=PRODUCTION_APP_ANALYSIS_ID,
    test_app_analysis_id=TEST_APP_ANALYSIS_ID,
    perspective_code=PERSPECTIVE_CODE,
    include_plt=INCLUDE_PLT,
    relative_tolerance=RELATIVE_TOLERANCE,
)

print("Validation complete.")

Validation complete.


## 4. Validation Results

In [4]:
print_validation_summary(result)

ANALYSIS VALIDATION RESULTS

Production Analysis ID: 1665
Test Analysis ID:       4419
Perspective:            GR

------------------------------------------------------------
Endpoint Results:
------------------------------------------------------------
  [OK] Statistics: PASS
  [OK] EP Metrics: PASS
  [OK] ELT: PASS

OVERALL: PASS


## 5. Detailed Differences (if any)

In [5]:
print_validation_details(result, max_differences=MAX_DIFFERENCES_TO_SHOW)


No differences found - all endpoints match!
